# RKDE (Robust Kernel Density Estimation)

## Generating  the data set

In [22]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams['figure.figsize'] = (20,10)
def pdf(loc,variance,observation):
    '''Giving the Probability Density Function

    Parameters
    ----------
        loc (Int): Mean of the distribution ; where the peak of the bell exists
        variance(Int) : variance
        sample (Array) : the distribution

    Returns
    -------
        ndrray: Gaussian distribution
    '''
    # A normal continuous random variable.
    s1 = 1/(np.sqrt(2*np.pi*variance))
    s2 = np.exp(-(np.square(observation - loc)/(2*variance)))
    return s1 * s2 

# Define the number of points
n_samples = 200

mu1, sigma1 = 0,1 # mean and variance
mu2, sigma2 = 15,1

x1 = np.random.normal(mu1,np.sqrt(sigma1),n_samples)
x2 = np.random.normal(mu2,np.sqrt(sigma2),n_samples)
# Adding outliers
outliers1 = np.random.uniform(5.5,6,25)
outliers2 = np.random.uniform(9,10,25)
X = np.array(list(x1) + list(x2))
outliers = np.append(outliers1, outliers2)
X = np.append(X, outliers)
#np.random.shuffle(X)
print("Dataset Shape: ", X.shape)
n_samples_final = X.shape[0]
# STANDARD DEVIATION
std = np.std(X)
# WINDOWS SIZE
#windows = (1/np.sqrt(n_samples_final)*20)
windows = 1.06 * n_samples_final**(-1/5) * std

Dataset Shape:  (450,)


## KDE functions

In [23]:
def kernel_function(obs,givenData,h,d=1):
    """ Kernel Function

    Parameters
    ----------
        obs (Int): Observation data
        h (Int): variance of the distribution
        d (Int, optional): dimension
        givenData (Int): Gaussian value
    
    Returns
    -------
        Int: kernel value   
    """
    result = (h**2 * 2*np.pi)**(-d/2) *np.exp((-1/2)* ((obs - givenData)/h)**2)
    return result

def kernel_density_function(obs_data,givenData,h=.1,d=1):
    '''Kernel Density Function
    Parameters
    ----------
        obs_data (Array): Observation data  
        h (Int): windows size
        d (Int, optional): dimension
        givenData (Array): Gaussian values

    Returns
    -------
      Array: new data after applying kdf
    '''
    final_result = []
    # Size of the gaussian data
    size = len(givenData)
    for obs in obs_data:
        k_result = 0
        for g in givenData:
            k_result +=  (1/size) * kernel_function(obs,g,h)
        final_result.append(k_result)
    return final_result

def kernel_density_function_for_rkde(obs_data,givenData,weights, h=.1,d=1):
    '''Kernel Density Function
    Parameters
    ----------
        obs_data (Array): Observation data  
        h (Int): windows size

        d (Int, optional): dimension
        givenData (Array): Gaussian values

    Returns
    -------
      Array: new data after applying kdf
    '''
    final_result = []
    #weights = np.array(weights)
    #weights = np.array(weights)
    # Size of the gaussian data
    size = len(givenData)
    for obs in obs_data:
        k_result = 0
        for index,g in enumerate(givenData):
            #print(index)
            k_result +=  weights[index] * kernel_function(obs,g,h)
        final_result.append(k_result)
    return final_result 
# WINDOWS SIZE
windows = (1/np.sqrt(n_samples_final)*20)

In [24]:
from collections import Counter

def search_weight(X):
    count_weights = Counter(X)
    total = sum(count_weights.values())
    w_0 = [(count_weights[w] / total) for w in X]
    #print(count_weights)
    return w_0

def operand_2_func(index,weights,X,size_dist,windows):
    '''Generate operand 2

    Parameters
    ----------
        weights (array): the weights  
        X (array): the distribution
        windows (int): Windows size

    Returns
    -------
      int: operand 2
    '''
    #result = []
    result = 0
    for j in range(size_dist):
       #result.append(-2*weights[int(X[j])]*kernel_function(X[index],X[j],windows))
       result += weights[int(X[j])]*kernel_function(X[index],X[j],windows)
    result *= -2   
       #print(result)
    return result

def operand_3_func(weights,X, size_dist,windows):

  '''Generate operand 3

    Parameters
    ----------
        weights (array): the weights  
        X (array): the distribution
        windows (int): Windows size
    Returns
    ----------
      int: operand 3
    '''
  #final_result = []
  final_result = 0
  for i in range(size_dist):
    result = 0
    for j in range(size_dist):
      result += weights[int(X[i])]*weights[int(X[j])]*kernel_function(X[i],X[j],windows)
    #final_result.append(result)
    final_result += result
  return final_result

def huber_loss_func(value):
    alpha = 0.05
    #alpha = .0022
    #print(value)
    if value < -alpha:
        return -alpha
    elif -alpha <= value and value <= alpha:
        return value
    else:
        return alpha

def update_weights(value):
    #print("Huber loss")
    #print(huber_loss_func(value))
    return huber_loss_func(value) / value

#Step 4 : Normalize the updated weights
def normalize_weight(updated_weights,sum_weights):
    '''Normalize weight

    Parameters
    ----------
        updated_weights (int): the weights  
        sum_weights (array): summation of the initial weight

    Returns
    -------
      int:  normalized weight
    '''
    return updated_weights / sum_weights
    
#Step 5 : If the algorithm converges
def convergence(w_0,normalized_weights):
    """ powered_w_0 = [x**2 for x in w_0]
    powered_normalized_weights = [x**2 for x in normalized_weights]
    convergence = (np.dot(w_0,normalized_weights)) / (np.sqrt(sum(powered_w_0)) * np.sqrt(sum(powered_normalized_weights))) """
    power_difference = np.power(np.subtract(w_0,normalized_weights),2)
    convergence = np.sqrt(sum(power_difference))
    return convergence

In [25]:
w_0 = search_weight(X)
print("100 First weight of the distribution\n")
print(w_0[:10])
print("\nWeights of the outliers")
print(w_0[:10])
print(sum(w_0))

100 First weight of the distribution

[0.0022222222222222222, 0.0022222222222222222, 0.0022222222222222222, 0.0022222222222222222, 0.0022222222222222222, 0.0022222222222222222, 0.0022222222222222222, 0.0022222222222222222, 0.0022222222222222222, 0.0022222222222222222]

Weights of the outliers
[0.0022222222222222222, 0.0022222222222222222, 0.0022222222222222222, 0.0022222222222222222, 0.0022222222222222222, 0.0022222222222222222, 0.0022222222222222222, 0.0022222222222222222, 0.0022222222222222222, 0.0022222222222222222]
1.000000000000004


## Applying the process 

In [26]:

""" def process(X, outliers, w_0, size_sample, windows, k = 1,alpha=.6):
    ## STEP 1
    
    ## Step 2
    operand_1 = kernel_function(X[k-1],X[k-1],windows)
    operand_2 = operand_2_func(k-1,w_0,X,size_sample,windows)
    operand_3 = operand_3_func(w_0,X,size_sample,windows)
    step2_result = np.sqrt(operand_1 + operand_2 + operand_3)

    ## Step 3
    updated_weights = [update_weights(value) for value in step2_result]
    
    ## Step 4
    normalized_weights = [normalize_weight(weight,sum(updated_weights)) for weight in updated_weights]
    #print(normalized_weights[-10:])
    ## Step 5
    #print(w_0[:100])
    #print("\n")
    #print(normalized_weights)
    convergence_val = convergence(w_0,normalized_weights)
    print("Convergence")
    print(convergence_val)
    #print(X.shape[0])
    #print(normalized_weights)

    if (np.round(convergence_val,3) > alpha):
        w_0 = normalized_weights
        print("\nNew weights outliers")
        print(w_0[-10:])

        k += 1
        print("iteration")
        print(k)
        process(X, outliers, w_0, size_sample, windows, k, alpha)
    else:
        print("else")
        new_X = np.random.choice(X,n_samples_final,p=normalized_weights)
        X_rkde = kernel_density_function_for_rkde(bins,new_X,normalized_weights,windows)
        ##### PLOTING ##########
        plt.xlabel("$x$")
        plt.ylabel("pdf")

        plt.scatter(X,[.005] * len(X), color='navy', s=30, marker=2, label="Train Data")
        #plt.scatter(outliers,[.005] * len(outliers), color='red', s=20, marker='x', label="Outliers")

        plt.plot(bins, .5 * pdf(mu1,sigma1,bins)+ .5 * pdf(mu2,sigma2,bins), color='black', label="True")
        #plt.plot(bins, pdf(mu2,sigma2,bins), color='black')

        plt.plot(bins, X_kde, color='blue', label="KDE", dashes=[6,2])
        plt.plot(bins, X_rkde, color='red', label="RKDE", dashes=[6,5])
        plt.legend()
        plt.show() """

#process(X, outliers,w_0,n_samples_final,windows)      

' def process(X, outliers, w_0, size_sample, windows, k = 1,alpha=.6):\n    ## STEP 1\n    \n    ## Step 2\n    operand_1 = kernel_function(X[k-1],X[k-1],windows)\n    operand_2 = operand_2_func(k-1,w_0,X,size_sample,windows)\n    operand_3 = operand_3_func(w_0,X,size_sample,windows)\n    step2_result = np.sqrt(operand_1 + operand_2 + operand_3)\n\n    ## Step 3\n    updated_weights = [update_weights(value) for value in step2_result]\n    \n    ## Step 4\n    normalized_weights = [normalize_weight(weight,sum(updated_weights)) for weight in updated_weights]\n    #print(normalized_weights[-10:])\n    ## Step 5\n    #print(w_0[:100])\n    #print("\n")\n    #print(normalized_weights)\n    convergence_val = convergence(w_0,normalized_weights)\n    print("Convergence")\n    print(convergence_val)\n    #print(X.shape[0])\n    #print(normalized_weights)\n\n    if (np.round(convergence_val,3) > alpha):\n        w_0 = normalized_weights\n        print("\nNew weights outliers")\n        print(w_0

In [44]:
def process(X, w_0, windows, k=1, alpha=.001,iterations={}):
    #new_weights = []
    updated_weights = []
    step2_result = []
    length = len(X)
    for index in range(length):
        ## Step 2
        operand_1 = kernel_function(X[index],X[index],windows)
        operand_2 = operand_2_func(index,w_0,X,length,windows)
        operand_3 = operand_3_func(w_0,X,length,windows)
        result = np.sqrt(operand_1 + operand_2 + operand_3)
        step2_result.append(result) 
        ## Step 3   
        updated_weights.append(update_weights(result))
    ## Step 3
    """ for value in step2_result:
        updated_weights.append(update_weights(value)) """
    
    print("Summation of weights after updating {}".format(sum(updated_weights)))
    print(updated_weights)
    ## Step 4
    normalized_weights = [normalize_weight(weight,sum(updated_weights)) for weight in updated_weights]
    """ print(normalized_weights[:10])
    print(sum(normalized_weights)) """
    ## Step 5
    #print("\n")
    #print(normalized_weights[:100])
    convergence_val = convergence(w_0,normalized_weights)
    print("Convergence")
    print(convergence_val)
    #print(X.shape[0])
    #print(normalized_weights)
    iterations[k] = normalized_weights
    if(k<15):
    ##if (np.round(convergence_val,3) > alpha):
        print("iteration")
        
        """ print(k)
        print(w_0[10:-10]) """
        w_0 = normalized_weights
        #print("\nNew Weights Outliers")
        #print(w_0[:-50])
        k = k + 1
        print("New K")
        print(k)
        """ if k==101 :
            return """
        process(X, w_0, windows,k,iterations=iterations)
    else:
        print("INTERATIONS AND NORMALIZED WEIGHT")
        print(iterations)
        n_samples_final = X.shape[0]
        bins = np.linspace(np.min(X),np.max(X),n_samples_final)
        # Calculation of the KDE
        #X_kde = kernel_density_function(bins,X,windows)
        #new_X = np.random.choice(X,n_samples_final,p=normalized_weights)
        # Calculation of the RKDE with the new weights
        #X_rkde = kernel_density_function_for_rkde(bins,X,normalized_weights,windows)
        ##### PLOTING ##########
        #plt.xlabel("$x$")
        #plt.ylabel("pdf")

        #plt.scatter(X,[.005] * len(X), color='navy', s=30, marker=2, label="Train Data")
        #plt.scatter(outliers,[.005] * len(outliers), color='red', s=20, marker='x', label="Outliers")

        #plt.plot(bins, .5 * pdf(mu1,sigma1,bins)+ .5 * pdf(mu2,sigma2,bins), color='black', label="True")
        #plt.plot(bins, pdf(mu2,sigma2,bins), color='black')

        #plt.plot(bins, X_kde, color='blue', label="KDE", dashes=[6,2])
        #plt.plot(bins, X_rkde, color='red', label="RKDE", dashes=[6,5])
        #plt.legend()
        #plt.show()
        
    return iterations    

""" for i in range(len(X)):
    new_weights.append(process(i,X,w_0,windows))"""


#print(sum(weights[-20:]))
#print("\n")
#print(sum(new_weights[-20:]))

' for i in range(len(X)):\n    new_weights.append(process(i,X,w_0,windows))'

In [45]:
record_weights = process(X, w_0, windows)

Summation of weights after updating 40.080667783679445
[0.0945970662998479, 0.08252242918933762, 0.08562561027324528, 0.09814382892188986, 0.08064730368220784, 0.09818530632403183, 0.09437427402455248, 0.08386938580434454, 0.09808325635668048, 0.09811794363441458, 0.09781063529653516, 0.09815638757134386, 0.08735905648804093, 0.07724983236522996, 0.07926483065850791, 0.08866173180924168, 0.09796101982356581, 0.09818745991856247, 0.08985372502976184, 0.09226937303547293, 0.09186267418447845, 0.07252633483104373, 0.094952723632825, 0.08116692787943156, 0.0947101119518119, 0.09730776315292577, 0.09058476607727084, 0.08761242087816062, 0.07443468536322387, 0.08681147410948448, 0.09550465722975089, 0.09702163598712754, 0.0933713128681715, 0.07512785590013821, 0.09768416317651672, 0.0913498050805047, 0.09814950249030355, 0.09816202662909768, 0.09731127450103909, 0.0954937907720401, 0.08746005117533015, 0.0900618860197171, 0.09643264356418624, 0.09191390937164903, 0.08203086981859821, 0.09086

In [52]:
print(record_weights[1][-50:])

[0.0018119765844306435, 0.001811916853099843, 0.0018103744627985165, 0.001812225398366918, 0.001811068883820817, 0.0018123590405813913, 0.0018125858035226371, 0.001810795651188955, 0.0018103496745317913, 0.0018103794304841787, 0.001811058771947473, 0.0018125842884957027, 0.0018122881932972645, 0.001810252284169303, 0.0018121698065943267, 0.0018105529723113922, 0.001812585919079728, 0.001812231356537183, 0.0018109898799111694, 0.0018108680455892886, 0.0018116509201772943, 0.0018114409364349956, 0.0018107610773177864, 0.001812504623494532, 0.0018107790969248728, 0.0018090119520902336, 0.001805827639313227, 0.001805105790800031, 0.001808715870275815, 0.0018040553558248396, 0.0018058968384332748, 0.001809216022085285, 0.0018034906479292368, 0.0018037112507401144, 0.001807402061250826, 0.0018074960074366134, 0.0017997605038976058, 0.0018057261753070665, 0.001806794351668317, 0.001798453952530318, 0.0018038801548814398, 0.0018088985325591261, 0.0018034918623199162, 0.001808523353929201, 0.00

In [53]:
print(record_weights[2][-50:])

[0.001824007754352095, 0.0018239405423424939, 0.0018222057068586984, 0.0018242877815549511, 0.0018229919913493318, 0.0018244382452221187, 0.001824694157397378, 0.0018226852375807384, 0.0018221845544233199, 0.0018222112929566223, 0.0018229806389075344, 0.0018246924897658415, 0.0018243607052570777, 0.001822068320673599, 0.0018242278597230997, 0.0018224064428144227, 0.0018246941519054424, 0.0018242944885555569, 0.0018228977913121735, 0.0018227607695055759, 0.001823645421048527, 0.0018234096854289914, 0.0018226464222028396, 0.0018246034237586546, 0.0018226666524407257, 0.0018206046437642218, 0.0018170170862495166, 0.0018162355960806371, 0.00182027950058596, 0.0018150587462243519, 0.0018170949847279574, 0.0018208370567124165, 0.0018143867075694998, 0.0018146349813213235, 0.0018187897760811717, 0.0018188955850866018, 0.0018102486915568755, 0.0018169306848736901, 0.0018181054364221666, 0.0018087860509293072, 0.0018148250740021635, 0.0018204837481935118, 0.0018143880742764087, 0.00182005318908